### 0. Import nltk

In [3]:
%pip install nltk # uncomment this for first time use
%pip install pandas
import nltk
nltk.download()

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 435.7 kB/s eta 0:00:27
   ---------------------------------------- 0.0/11.5 MB 178.6 kB/s eta 0:01:05
   ---------------------------------------- 0.1/11.5 MB 261.4 kB/s eta 0:00:44
   ---------------------------------------- 0.1/11.5 MB 294.4 kB/s eta 0:00:39
   ---------------------------------------- 0.1/11.5 MB 274.5 kB/s eta 0:00:42
    --------------------------------------- 0.2/11.5 MB 296.6 kB/s eta 0:00:39
    --------------------------------------- 0.3/11.5 MB 334.4 kB/s eta 0:00:34
    --------------------------------------- 0.3/11.5 MB 354.0 kB/s eta 0:00:32
   - -------------------------------------- 0.3/11.5 MB 351.1 kB/s eta 0:00:32
   - -------------------------------------- 0.3/11.5 MB 351.1 kB/s eta 0:00:32
   - -------------------------------------- 0.3/11.5 MB 354.6 kB/s et


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### 1. Data Pre-processing

In [4]:
import pandas as pd

df = pd.read_csv('../data/raw.csv')

Create an empty list for each mental disorder

In [5]:
depression = list()
personality_disorder = list()
bipolar = list()
anxiety = list()
stress = list()
normal = list()
suicide = list()

Append the data to each list

In [6]:
for i in range(len(df.status)):
    if df.status[i] == "Depression":
        depression.append(df.statement[i])
    elif df.status[i] == "Personality disorder":
        personality_disorder.append(df.statement[i])
    elif df.status[i] == "Bipolar":
        bipolar.append(df.statement[i])
    elif df.status[i] == "Anxiety":
        anxiety.append(df.statement[i])
    elif df.status[i] == "Stress":
        stress.append(df.statement[i])
    elif df.status[i] == "Normal":
        normal.append(df.statement[i])
    elif df.status[i] == "Suicidal":
        suicide.append(df.statement[i])
    else:
        continue

Tokenizing

In [7]:
from nltk.tokenize import word_tokenize

list_words = word_tokenize("".join(str(depression))) + word_tokenize("".join(str(personality_disorder))) + word_tokenize("".join(str(bipolar))) + word_tokenize("".join(str(anxiety))) + word_tokenize("".join(str(stress))) + word_tokenize("".join(str(normal))) + word_tokenize("".join(str(suicide)))

Words cleaning & filtering

In [8]:
from nltk.corpus import stopwords
from string import punctuation

eng_stopwords = stopwords.words('english')

list_words = [word for word in list_words if word not in eng_stopwords]
list_words = [word for word in list_words if word not in punctuation]
list_words = [word for word in list_words if word.isalpha()]

len(list_words)

3029791

Smalling Range

In [10]:
from nltk.probability import FreqDist

fd = FreqDist(list_words)

list_words = [word for word, count in fd.most_common(20000)]

len(list_words)

20000

Labeling

In [11]:
labeled_sentences = []
for sentence in depression:
    labeled_sentences.append((sentence, "Depression"))

for sentence in personality_disorder:
    labeled_sentences.append((sentence, "Personality Disorder"))

for sentence in bipolar:
    labeled_sentences.append((sentence, "Bipolar"))

for sentence in anxiety:
    labeled_sentences.append((sentence, "Anxiety"))

for sentence in stress:
    labeled_sentences.append((sentence, "Stress"))

for sentence in normal:
    labeled_sentences.append((sentence, "Normal"))

for sentence in suicide:
    labeled_sentences.append((sentence, "Suicidal"))

### 2. Dataset Making

In [12]:
dataset = []
for sentence, label in labeled_sentences:
    dict = {}
    words = word_tokenize(str(sentence))
    for feature in list_words:
        key = feature
        value = feature in words
        dict[key] = value
    dataset.append((dict, label))

import random
random.shuffle(dataset)

train_set = dataset[:int(len(dataset) * 0.7)]
test_set = dataset[int(len(dataset) * 0.3):]

In [13]:
from nltk.classify import NaiveBayesClassifier, accuracy
classifier = NaiveBayesClassifier.train(train_set)
accuracy(classifier, test_set)

0.6412065715055212

In [14]:
import pickle
file = open("classifier.pickle", "wb")
pickle.dump(classifier, file)